<a href="https://colab.research.google.com/github/mapbefun/mapbefun/blob/main/MNIST%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%95%99%EC%8A%B5%EA%B3%BC%EA%B2%80%EC%A6%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

제 7장 합성곱 신경망(CNN)을 듣고 아래를 파이썬을 이용하여 구현하고 결과를 분석하시오.<br>
Keras를 이용하여 동영상과 동일하거나 자신이 생각하는 CNN을 구현하시오. (제7장_합성곱 신경망(CNN)_02 동영상 참조) <br>
회전을 이용하여 데이터를 확장한 후, 다시 학습시킨 후, 검증용 데이터를 이용한 정확도를 계산한다.(우리가 만든 MNIST 검증데이터와 MNIST 검증데이터 각각의 정확도를 계산) <br>
확장한 데이터와 MNIST데이터를 통합한 후, 다시 학습시킨 후, 검증용 데이터를 이용한 정확도를 계산한다.(우리가 만든 MNIST 검증데이터와 MNIST 검증데이터 각각의 정확도를 계산)<br>

In [ ]:
# 데이터셋 불러오기.
from google.colab import drive
drive.mount('/datalab/drive')

In [ ]:
import sys, os
sys.path.append(os.pardir)
sys.path.append('/datalab/drive/MyDrive/딥러닝수업')
my_path = '/datalab/drive/MyDrive/딥러닝수업/'

In [ ]:
# MNIST 데이터셋 입력.
import pickle

with open(my_path+"OurMNIST/OurMNISTdata.pkl","rb") as f:
  our_images=pickle.load(f)
with open(my_path+"OurMNIST/OurMNISTlabel.pkl","rb") as f:
  our_labels=pickle.load(f)

print(our_images.shape)
print(our_labels.shape)

In [ ]:
# One_Hot_Encoded data 정의.
def one_hot_encoder(data,nclass):
  rows = np.arange(data.shape[0])
  one_hot = np.zeros((data.shape[0],nclass),dtype = "int32")
  one_hot[rows,data] = 1
  return one_hot

In [ ]:
#넘파이 라이브러리 불러오기
import numpy as np
our_labels=one_hot_encoder(our_labels,10)

In [ ]:
# 우리 숫자 데이터에 NaN값 제거

our_images_1d = our_images.reshape(-1,28*28)
pos=[]
for i, img in enumerate(our_images_1d):
    if np.sum(np.isnan(img)):
        pos.append(i)

if pos !=[]:
    our_images_1d= np.delete(our_images_1d,np.array(pos),axis=0)
    our_labels= np.delete(our_labels,np.array(pos),axis=0)

    print(our_images_1d.shape)
    print(our_labels.shape)

In [ ]:
# 훈련용데이터 80%, 검증용 데이터는 20% 만들기.
Ntrain = int(our_images_1d.shape[0]*0.8)
Ntest = our_images_1d.shape[0] - Ntrain

#data를 reshape해준다.
our_images = our_images_1d.reshape(-1,28,28,1) #(원래있는데이터갯수,가로크기,세로크기,밴드갯수)

x_train = our_images[:Ntrain]
t_train = our_labels[:Ntrain]
x_test = our_images[Ntrain:]
t_test = our_labels[Ntrain:]

print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

In [ ]:
# 이미지를 회전 시켜서 데이터 확장
import skimage. transform as transform

x_train2 = np.zeros((x_train.shape[0]*5,28,28,1))
t_train2 = np.zeros((t_train.shape[0]*5,10))

for i , img in enumerate(x_train):
  x_train2[5*i] = x_train[i]
  x_train2[5*i+1] = transform.rotate(x_train[i].reshape(28,28), angle = 8, resize=False).reshape(28,28,1)
  x_train2[5*i+2] = transform.rotate(x_train[i].reshape(28,28), angle = -8, resize=False).reshape(28,28,1)
  x_train2[5*i+3] = transform.rotate(x_train[i].reshape(28,28), angle = 16, resize=False).reshape(28,28,1)
  x_train2[5*i+4] = transform.rotate(x_train[i].reshape(28,28), angle = -16, resize=False).reshape(28,28,1)

  t_train2[5*i]     = t_train[i]
  t_train2[5*i+1]   = t_train[i]
  t_train2[5*i+2]   = t_train[i]
  t_train2[5*i+3]   = t_train[i]
  t_train2[5*i+4]   = t_train[i]

In [ ]:
# Keras로 구현
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

In [ ]:
def CnnKeras(input_size=(28,28,1)):
    in1 = input(shape=input_size)

    x = Conv2D(128,(3,3),kernel_initializer='he_normal',padding="same")(in1)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Conv2D(32,(3,3),kernel_initializer='he_normal',padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Conv2D(8,(3,3),kernel_initializer='he_normal',padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x) #28,28을 2배로 줄여 14,14로 만들어 준다.

    x = Conv2D(128,(3,3),kernel_initializer='he_normal',padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Conv2D(32,(3,3),kernel_initializer='he_normal',padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Conv2D(8,(3,3),kernel_initializer='he_normal',padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x) #7,7

    #fully connected layer 연결
    x = Flatten()(x) #2차원을 1차원으로 만들어줌
    x = Dense(128,kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Dense(32,kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Dense(10,kernel_initializer='he_normal')(x)
    x = Activation('softmax')(x)

    model = Model(inputs=[in1],outputs=[x])

    return model 


In [ ]:
# 모델컴파일
lr = 0.001
mini_batch = 100
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["gpu:0"])
with mirrored_strategy.scope():
  model = CnnKeras()
  model.compile(optimizer=Adam(learning_rate=lr),loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])

model.summary()